# Homework 3

## Problem 1

It could be the case that just by chance, the generated validation set correspondes perfectly to the trained model, meaning that the validation didn't represent the true performance of the model.

## Problem 2

According to the lecture note we have

$$
\text{P} \left[ \underset{k=1,\cdots,K}{\text{max}} \left| \hat{R} \left(h_k, \mathcal{D}_{val} \right) - R\left( h_k \right) \right| \le O\left( \sqrt{\frac{\text{log} \left( K / \delta \right)}{\left| \mathcal{D}_{val} \right|}} \right) \right] \ge 1- \delta ,
$$

to correctly chose a $k$ without runging into problem, we can look at the term

$$
\sqrt{\frac{\text{log} \left( K / \delta \right)}{\left| \mathcal{D}_{val} \right|}},
$$

to obtain a validation error that gives a good estimate of the risk for all hyperparameter configurations, $\text{log}(K)$ should be small relative to $\left| \mathcal{D}_{val} \right|$, i.e.,

$$
\text{log}(K) \le \left| \mathcal{D}_{val} \right| ~~~~~~ \Rightarrow ~~~~~~ K \le e^{\left| \mathcal{D}_{val} \right|} .
$$

Let the $\left| \mathcal{D}_{val,small} \right|$ for the small validation set be $a$ and therefore for the large validation set $\left| \mathcal{D}_{val,large} \right| = 10 \cdot a$. On the small validation set we can evaluate $e^a$ models and on the large validation set, $e^{10 \cdot a}$ models can be tested.

## Problem 3

### 1.

Some features have larger deviation because of their nature, and during the ridge regression, these variables will receive larger penalty, which makes them more "important" than they should be.

By scaling the variables, the effect of different variables on the final model is normalized.

### 2.

We can define the feature vector as $\mathbf{\theta}_{\text{ridge}} = \left[\theta_0, \theta_1, \cdots, \theta_N \right] = \left[ 0, \mathbf{\theta} \right] \in \mathbb{R}^{N+1}$. Together with the given definition of $\tilde{\mathbf{X}}$, we have traditional ridge regeression as follows:

$$
\begin{alignat*}{2}
  \hat{\mathbf{\theta}}_{\text{ridge}} &= \text{arg} \underset{\mathbf{\theta}_{\text{ridge}}}{\text{min}} \left\| \mathbf{y} - \tilde{\mathbf{X}} \mathbf{\theta}_{\text{ridge}} \right\|_2^2 + \lambda \left\| \mathbf{\theta}_{\text{ridge}} \right\|_2^2 \\
  &= \text{arg} \underset{\mathbf{\theta}_{\text{ridge}}}{\text{min}} \left\| \mathbf{y} - \tilde{\mathbf{X}} \mathbf{\theta}_{\text{ridge}} \right\|_2^2 + \lambda \mathbf{\theta}_{\text{ridge}}^\top \mathbf{\theta}_{\text{ridge}}.
\end{alignat*}
$$

According to the task, $\theta_0$ should not be considered for the penalty, to achieve this, we can multiply the penalty term with an identity, but starting with 0, denoted as $\mathbf{A}$ i.e.,

$$
\mathbf{A} = \begin{bmatrix}
0 &                         \\
  & \mathbf{I}_{N \times N}
\end{bmatrix} \in \mathbb{R}^{N+1 \times N+1},
$$

by doing this, the ridge regression problem can be rewritten as

$$
\hat{\mathbf{\theta}}_{\text{ridge}} = \text{arg} \underset{\mathbf{\theta}_{\text{ridge}}}{\text{min}} \left\| \mathbf{y} - \tilde{\mathbf{X}} \mathbf{\theta}_{\text{ridge}} \right\|_2^2 + \lambda \mathbf{\theta}_{\text{ridge}}^\top \mathbf{A} \mathbf{\theta}_{\text{ridge}}.
$$

We can replace the identity term $\mathbf{I}$ in closed-form solution of ridge regression with the newly defined $\mathbf{A}$, leading to

$$

$$

### 3.

In [36]:
import csv
import numpy as np
from matplotlib import pyplot as plt

# Load features
X = np.loadtxt("hitters/hitters.x.csv", delimiter = ",", skiprows = 1)
with open("hitters/hitters.x.csv", "r") as f:
  X_colnames = next(csv.reader(f))

# Load salaries
y = np.loadtxt("hitters/hitters.y.csv", delimiter = ",", skiprows = 1)

# Normailize the feature vectors as discussed in subtask 1
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

array([[-0.6029005 , -0.59567545, -0.5285512 , ..., -0.52319133,
         0.21335208,  1.07505242],
       [ 0.51254171,  0.49225957,  0.72996619, ..., -0.25386267,
         0.81996395, -0.9301872 ],
       [ 0.62816682,  0.73648988,  0.95878753, ..., -0.74417894,
        -0.84821868,  1.07505242],
       ...,
       [ 0.48533581,  0.40344855, -0.98619389, ..., -0.03978092,
        -0.24160682, -0.9301872 ],
       [ 1.15188054,  0.80309815, -0.29972986, ...,  0.08452461,
         0.51665801, -0.9301872 ],
       [ 1.54636621,  1.38036979, -0.29972986, ..., -0.79251998,
        -0.84821868, -0.9301872 ]])